# Superstore Dataset - Data Cleaning and Preprocessing

In this notebook, we focus on data preparation, cleaning, and preprocessing for the Superstore dataset, a comprehensive dataset often used for sales analysis, customer segmentation, and profit prediction tasks based on various order, product, and customer attributes.

Good data preprocessing is crucial for reliable and interpretable results in business intelligence and analytics workflows. Here, I address common data issues such as missing values, duplicates, and inconsistent categorical labels, while creating derived features to improve downstream analysis.

I start by importing essential Python libraries for data handling and manipulation.

- `pandas` for structured data operations.

- `numpy` for numerical operations.

- `os` for interacting with the operating system and directory structures.

In [3]:
import pandas as pd
import numpy as np
import os
import io

## Define and Create Directory Paths

To ensure reproducibility andorganized storage, we programmatically create directories for:

- **raw data**
- **processed data**
- **results**
- **documentation**

These directories will store intermediate and final outputs for reproducibility.

# Define and Create Paths

In [4]:
# Get current working directory
current_dir = os.getcwd()

# Go one directory up (assuming script is inside a subfolder like 'notebooks')
project_root_dir = os.path.dirname(current_dir)

# Define key folder paths
data_dir = os.path.join(project_root_dir, 'data')
raw_dir = os.path.join(data_dir, 'raw')
processed_dir = os.path.join(data_dir, 'processed')
results_dir = os.path.join(project_root_dir, 'results')
docs_dir = os.path.join(project_root_dir, 'docs')

# Create directories if they don't exist
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)
os.makedirs(docs_dir, exist_ok=True)


## Load Datasets
Three key datasets—'Orders', 'Returns', and 'People'—are loaded from the Superstore.xlsx Excel file into separate pandas DataFrames.

In [5]:
# Define the full path to your Excel file
excel_file_path = os.path.join(raw_dir, "Superstore.xlsx")

# Load the individual sheets
orders_df = pd.read_excel(excel_file_path, sheet_name='Orders')
returns_df = pd.read_excel(excel_file_path, sheet_name='Returns')
people_df = pd.read_excel(excel_file_path, sheet_name='People')

 
print("\nOrders DataFrame Head:")
print(orders_df.head())

 
print("\nReturns DataFrame Head:")
print(returns_df.head())

 
print("\nPeople DataFrame Head:")
print(people_df.head())



Orders DataFrame Head:
   Row ID        Order ID Order Date  Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156 2016-11-08 2016-11-11    Second Class    CG-12520   
1       2  CA-2016-152156 2016-11-08 2016-11-11    Second Class    CG-12520   
2       3  CA-2016-138688 2016-06-12 2016-06-16    Second Class    DV-13045   
3       4  US-2015-108966 2015-10-11 2015-10-18  Standard Class    SO-20335   
4       5  US-2015-108966 2015-10-11 2015-10-18  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \
0       

# Data Cleaning


## 1.  Data Merging
This section focuses on integrating the loaded datasets to create a unified DataFrame.

### Merge Orders and Returns
The 'Orders' DataFrame is merged with the 'Returns' DataFrame using a left join on 'Order ID'. This ensures that all order records are retained, and return information is added where available.

In [6]:
# Merge returns into orders (left join to keep all orders)
merged_df = pd.merge(orders_df, returns_df, on='Order ID', how='left')
merged_df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Returned
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,NaN
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,NaN
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,NaN
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,NaN
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028,NaN
9990,9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332,Yes
9991,9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932,Yes
9992,9993,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200,Yes


## Merge with People Data: 
The resulting merged DataFrame is then further merged with the 'People' DataFrame. This merge is performed using a left join on the 'Region' column, associating sales representatives with their respective regions.

In [7]:
# Merge the result with people data (left join to preserve all order records)
final_merged_df = pd.merge(merged_df, people_df, on='Region', how='left')
final_merged_df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Returned,Person
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,NaN,Cassandra Brandow
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,NaN,Cassandra Brandow
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,NaN,Anna Andreadi
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,NaN,Cassandra Brandow
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,NaN,Cassandra Brandow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028,NaN,Cassandra Brandow
9990,9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332,Yes,Anna Andreadi
9991,9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932,Yes,Anna Andreadi
9992,9993,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200,Yes,Anna Andreadi


In [8]:
final_merged_df.isnull().sum()

Row ID              0
Order ID            0
Order Date          0
Ship Date           0
Ship Mode           0
Customer ID         0
Customer Name       0
Segment             0
Country             0
City                0
State               0
Postal Code         0
Region              0
Product ID          0
Category            0
Sub-Category        0
Product Name        0
Sales               0
Quantity            0
Discount            0
Profit              0
Returned         9194
Person              0
dtype: int64

In [9]:
final_merged_df.head(10)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Returned,Person
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,NaN,Cassandra Brandow
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,NaN,Cassandra Brandow
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,NaN,Anna Andreadi
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,NaN,Cassandra Brandow
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,NaN,Cassandra Brandow
5,6,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694,NaN,Anna Andreadi
6,7,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.2800,4,0.00,1.9656,NaN,Anna Andreadi
7,8,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.1520,6,0.20,90.7152,NaN,Anna Andreadi
8,9,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by S...,18.5040,3,0.20,5.7825,NaN,Anna Andreadi
9,10,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9000,5,0.00,34.4700,NaN,Anna Andreadi


In [10]:
final_merged_df.shape

(9994, 23)

In [11]:
final_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9994 entries, 0 to 9993
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

## 2. Understanding the dataset
Before proceeding with the cleaning, we would like to understand the variables deeply. This would help guide the cleaning process. The subsequent tables detail the types, meaning, and values or ranges of the variables in the Superstore dataset.

**Table 1: Summary table of the variables in the dataset**


| Variable      | Type        | Description                          | Values / Range (excluding NaN)                                      |
| :------------ | :---------- | :----------------------------------- | :------------------------------------------------------------------ |
| Order ID      | Categorical | Unique identifier for each order     | Unique alphanumeric codes                                           |
| Order Date    | Date        | Date when the order was placed       | Dates ranging from 2014 to 2017                                     |
| Ship Date     | Date        | Date when the order was shipped      | Dates ranging from 2014 to 2017                                     |
| Ship Mode     | Categorical | Shipping method used                 | 'Second Class', 'Standard Class', 'First Class', 'Same Day'         |
| Customer ID   | Categorical | Unique identifier for each customer  | Unique alphanumeric codes                                           |
| Customer Name | Categorical | Name of the customer                 | Text names                                                          |
| Segment       | Categorical | Customer segment                     | 'Consumer', 'Corporate', 'Home Office'                              |
| Country       | Categorical | Country where the order was placed   | 'United States'                                                     |
| City          | Categorical | City where the order was placed      | Various city names (e.g., 'New York City', 'Los Angeles')           |
| State         | Categorical | State where the order was placed     | All 50 U.S. states and D.C.                                         |
| Postal Code   | Numeric     | Postal code of the delivery address  | 10001 – 99301                                                       |
| Region        | Categorical | Geographic region                    | 'East', 'Central', 'South', 'West'                                  |
| Product ID    | Categorical | Unique identifier for each product   | Unique alphanumeric codes                                           |
| Category      | Categorical | Main product category                | 'Furniture', 'Office Supplies', 'Technology'                        |
| Sub-Category  | Categorical | Sub-category of the product          | 'Bookcases', 'Chairs', 'Phones', 'Storage'                    |
| Product Name  | Categorical | Name of the product                  | Various product descriptions                                        |
| Sales         | Numeric     | Sales amount for the product         | 0.444 – 22,638.48                                                   |
| Quantity      | Numeric     | Quantity of the product ordered      | 1 – 14                                                              |
| Discount      | Numeric     | Discount applied to the product      | 0.0 – 0.8                                                           |
| Profit        | Numeric     | Profit generated from the product    | -6,599.978 – 8,399.976                                              |
| Returned      | Categorical | Indicates if the order was returned  | 'Yes', 'No'                                                         |
| Person        | Categorical | Sales manager responsible for region | 'Anna Andrus', 'Chuck Magee', 'Kelly Williams', 'Cassandra Brandow' |

**Table 2: Categorical Variables Table**

| Variable     | Unique Value             | Description                                                   |
| :----------- | :----------------------- | :------------------------------------------------------------ |
| Ship Mode    | Second Class             | Standard shipping, typically slower than First Class          |
|              | Standard Class           | Most common and often slowest shipping option                 |
|              | First Class              | Faster shipping option, quicker than Second Class             |
|              | Same Day                 | Fastest shipping option, delivery on the same day             |
| Segment      | Consumer                 | Individual customers purchasing for personal use              |
|              | Corporate                | Business customers, typically mid-sized companies             |
|              | Home Office              | Small business or work-from-home customers                    |
| Category     | Furniture                | Products related to furniture                                 |
|              | Office Supplies          | Products for office use                                       |
|              | Technology               | Electronic devices and related accessories                    |
| Returned     | Yes                      | The order was returned                                        |
|              | No                       | The order was not returned                                    |
| Person       | Anna Andrus              | Sales manager for a West region                               |
|              | Chuck Magee              | Sales manager for a East region                               |
|              | Kelly Williams           | Sales manager for a Central region                            |
|              | Cassandra Brandow        | Sales manager for a South region                              |
| State        | (Various US States)      | State where the order was placed (e.g., California, New York) |
| Region       | East                     | Orders from the Eastern United States                         |
|              | Central                  | Orders from the Central United States                         |
|              | South                    | Orders from the Southern United States                        |
|              | West                     | Orders from the Western United States                         |
| Sub-Category | (Various Sub-Categories) | Detailed product classifications (e.g., 'Phones', 'Binders')  |



In [12]:
print("\nUnique Ship Modes:")
print(np.unique(final_merged_df['Ship Mode'].dropna().to_list()))


Unique Ship Modes:
['First Class' 'Same Day' 'Second Class' 'Standard Class']


In [13]:
# Unique Segments
print("\nUnique Segments:")
print(np.unique(final_merged_df['Segment'].dropna().to_list()))


Unique Segments:
['Consumer' 'Corporate' 'Home Office']


In [14]:
# Unique Categories
print("\nUnique Categories:")
print(np.unique(final_merged_df['Category'].dropna().to_list()))


Unique Categories:
['Furniture' 'Office Supplies' 'Technology']


In [15]:
# Unique Regions
print("\nUnique Regions:")
print(np.unique(final_merged_df['Region'].dropna().to_list()))


Unique Regions:
['Central' 'East' 'South' 'West']


In [16]:
print("\nUnique Sub-Categories:")
print(np.unique(final_merged_df['Sub-Category'].dropna().to_list()))


Unique Sub-Categories:
['Accessories' 'Appliances' 'Art' 'Binders' 'Bookcases' 'Chairs' 'Copiers'
 'Envelopes' 'Fasteners' 'Furnishings' 'Labels' 'Machines' 'Paper'
 'Phones' 'Storage' 'Supplies' 'Tables']


In [17]:
print("\nUnique States:")
print(np.unique(final_merged_df['State'].dropna().to_list()))


Unique States:
['Alabama' 'Arizona' 'Arkansas' 'California' 'Colorado' 'Connecticut'
 'Delaware' 'District of Columbia' 'Florida' 'Georgia' 'Idaho' 'Illinois'
 'Indiana' 'Iowa' 'Kansas' 'Kentucky' 'Louisiana' 'Maine' 'Maryland'
 'Massachusetts' 'Michigan' 'Minnesota' 'Mississippi' 'Missouri' 'Montana'
 'Nebraska' 'Nevada' 'New Hampshire' 'New Jersey' 'New Mexico' 'New York'
 'North Carolina' 'North Dakota' 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania'
 'Rhode Island' 'South Carolina' 'South Dakota' 'Tennessee' 'Texas' 'Utah'
 'Vermont' 'Virginia' 'Washington' 'West Virginia' 'Wisconsin' 'Wyoming']


In [18]:
print("\nUnique Returned Statuses:")
print(np.unique(final_merged_df['Returned'].dropna().to_list()))


Unique Returned Statuses:
['Yes']


## 3. Deal with missing values

### Handle Missing Values
The 'Returned' column, which contained a significant number of missing values (NaN), is imputed by filling these entries with the string 'No'. This indicates that orders without a return record are considered not returned.

   ###  Replace NaN in 'Returned' column with 'No'

In [19]:
final_merged_df['Returned'] = final_merged_df['Returned'].fillna('No')
final_merged_df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Returned,Person
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,No,Cassandra Brandow
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,No,Cassandra Brandow
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,No,Anna Andreadi
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,No,Cassandra Brandow
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,No,Cassandra Brandow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028,No,Cassandra Brandow
9990,9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332,Yes,Anna Andreadi
9991,9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932,Yes,Anna Andreadi
9992,9993,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200,Yes,Anna Andreadi


## 3. Convert Data Types

 
### Create new feature: Shipping Duration in days
The 'Order Date' and 'Ship Date' columns are converted to datetime objects, enabling proper chronological analysis and operations.

In [20]:
final_merged_df['Shipping Duration'] = (
    final_merged_df['Ship Date'] - final_merged_df ['Order Date']
).dt.days
final_merged_df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Returned,Person,Shipping Duration
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,No,Cassandra Brandow,3
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,No,Cassandra Brandow,3
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,No,Anna Andreadi,4
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,No,Cassandra Brandow,7
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,No,Cassandra Brandow,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028,No,Cassandra Brandow,2
9990,9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332,Yes,Anna Andreadi,5
9991,9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932,Yes,Anna Andreadi,5
9992,9993,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200,Yes,Anna Andreadi,5


## 4. Extract order year and month for trend analysis
This step creates two new columns—Order Year and Order Month by extracting the year and month from the Order Date column using pandas' .dt accessor. These variables are useful for performing time-based trend analysis, such as identifying seasonal patterns or yearly growth in sales.

In [21]:
final_merged_df['Order Year'] = final_merged_df['Order Date'].dt.year
final_merged_df['Order Month'] = final_merged_df['Order Date'].dt.month
final_merged_df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Product Name,Sales,Quantity,Discount,Profit,Returned,Person,Shipping Duration,Order Year,Order Month
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,No,Cassandra Brandow,3,2016,11
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,No,Cassandra Brandow,3,2016,11
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,No,Anna Andreadi,4,2016,6
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,No,Cassandra Brandow,7,2015,10
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,No,Cassandra Brandow,7,2015,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,Ultra Door Pull Handle,25.2480,3,0.20,4.1028,No,Cassandra Brandow,2,2014,1
9990,9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332,Yes,Anna Andreadi,5,2017,2
9991,9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932,Yes,Anna Andreadi,5,2017,2
9992,9993,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200,Yes,Anna Andreadi,5,2017,2


## 5. Trim text columns of leading/trailing whitespace  
This step ensures the consistency and cleanliness of textual data by removing any unnecessary leading or trailing whitespace from string-type columns. This is a crucial universal cleanup practice that prevents issues during data analysis, filtering, or merging operations caused by subtle differences in string values due to whitespace.

### The process involves:

- Identifying Text Columns: All columns with an 'object' data type (typically representing strings) are selected from the final_merged_df.
- Applying Whitespace Trim: For each identified text column, the .str.strip() method is applied to every string entry. This method efficiently removes any spaces, tabs, or newlines from the beginning and end of the text, standardizing the data.

In [22]:
text_cols = final_merged_df.select_dtypes(include='object').columns
final_merged_df[text_cols] = final_merged_df[text_cols].apply(lambda x: x.str.strip())
final_merged_df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Product Name,Sales,Quantity,Discount,Profit,Returned,Person,Shipping Duration,Order Year,Order Month
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,No,Cassandra Brandow,3,2016,11
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,No,Cassandra Brandow,3,2016,11
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,No,Anna Andreadi,4,2016,6
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,No,Cassandra Brandow,7,2015,10
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,No,Cassandra Brandow,7,2015,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,Ultra Door Pull Handle,25.2480,3,0.20,4.1028,No,Cassandra Brandow,2,2014,1
9990,9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332,Yes,Anna Andreadi,5,2017,2
9991,9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932,Yes,Anna Andreadi,5,2017,2
9992,9993,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200,Yes,Anna Andreadi,5,2017,2


In [23]:
final_merged_df.isnull().sum()

Row ID               0
Order ID             0
Order Date           0
Ship Date            0
Ship Mode            0
Customer ID          0
Customer Name        0
Segment              0
Country              0
City                 0
State                0
Postal Code          0
Region               0
Product ID           0
Category             0
Sub-Category         0
Product Name         0
Sales                0
Quantity             0
Discount             0
Profit               0
Returned             0
Person               0
Shipping Duration    0
Order Year           0
Order Month          0
dtype: int64

## 6. Deal with Duplicates

Duplicate rows across the entire DataFrame are identified and removed to ensure data uniqueness and integrity. The process confirms that no duplicate entries remain after this operation, resulting in a cleaned DataFrame of (9994, 26) dimensions.

In [24]:
final_merged_df.duplicated().sum()

0

In [25]:
final_merged_df.shape

(9994, 26)

##  Save the Cleaned DataFrame to a CSV file

The final step involves persisting the cleaned and merged dataset for future use.

Export to CSV: The final_merged_df, now cleaned and preprocessed, is saved as 'final_superstore_cleaned.csv' within the designated 'processed data' directory. The 'index=False' argument ensures that the DataFrame index is not written to the CSV file.

In [27]:
# Define the full path for the output CSV
output_file = os.path.join(processed_dir, 'final_superstore_cleanedd.csv')

# Save the cleaned DataFrame
final_merged_df.to_csv(output_file, index=False)